In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### **Data Reading and Transformation**

In [0]:
df = spark.read.format("parquet")\
  .load("abfss://bronze@ayaneteprojstor.dfs.core.windows.net/products")

In [0]:
df.display()

In [0]:
df.createOrReplaceTempView("products")

### **Functions**

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_cata.bronze.discount_func(p_price DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL
RETURN p_price * 0.90

In [0]:
%sql
SELECT product_id, price, databricks_cata.bronze.discount_func(price) AS discounted_price
FROM products

In [0]:
df = df.withColumn("discounted_price", expr("databricks_cata.bronze.discount_func(price)"))
df.display()

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_cata.bronze.upper_func(p_brand STRING)
RETURNS STRING
LANGUAGE PYTHON
AS $$ return p_brand.upper() $$

In [0]:
%sql
SELECT product_id, brand, databricks_cata.bronze.upper_func(brand) AS brand_upper
FROM products

### **Data Writing**

In [0]:
df.write.format("delta").mode("overwrite")\
  .option("path","abfss://silver@ayaneteprojstor.dfs.core.windows.net/products")\
  .save()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cata.silver.products_silver
USING DELTA
LOCATION 'abfss://silver@ayaneteprojstor.dfs.core.windows.net/products'

In [0]:
%sql
SELECT * FROM databricks_cata.silver.products_silver